# Tarea 1 - Redes densas
## Eduardo García Alarcón

### 3. Red densa conl PyTorch

A partir de la libreta https://github.com/gibranfp/CursoAprendizajeProfundo/blob/2025-1/notebooks/1c_redes_densas_pytorch.ipynb, evalúa tres distintas configuraciones de redes densas que incluyan lo siguiente:

  - Más capas ocultas
  - Distintas funciones de activasión de las capas ocultas
  - Técnicas de regularización
  - Capas de normalizaciónl   